In [ ]:
# From https://www.kaggle.com/bhavikjain/tps-june-21-eda-models

In [4]:
!pip install tensorflow

In [5]:
!pip install tensorflow-addons

In [6]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import log_loss

import gc
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations,callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers

from keras.models import Model

In [7]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

X_dim = np.shape(train)[1] - 2 # id and target
print(X_dim)

75


In [8]:
targets = pd.get_dummies(train['target'])

In [9]:
def custom_metric(y_true, y_pred):
    y_pred = K.clip(y_pred, 1e-15, 1-1e-15)
    loss = K.mean(cce(y_true, y_pred))
    return loss

cce = tf.keras.losses.CategoricalCrossentropy()

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_custom_metric', min_delta=1e-05, patience=5, verbose=0,
    mode='min', baseline=None, restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_custom_metric', factor=0.7, patience=2, verbose=0,
    mode='min')

In [10]:
def conv_model():

    conv_inputs = layers.Input(shape = (X_dim))
    embed = layers.Embedding (input_dim = 354, 
                              output_dim = 7,
                              embeddings_regularizer='l2')(conv_inputs)
    embed = layers.Conv1D(12,1,activation = 'relu')(embed)        
    embed = layers.Flatten()(embed)
    hidden = layers.Dropout(0.3)(embed)
    
    hidden = tfa.layers.WeightNormalization(
                layers.Dense(
                units=32,
                activation ='selu',
                kernel_initializer = "lecun_normal"))(hidden)
    
    output = layers.Dropout(0.3)(layers.Concatenate()([embed, hidden]))
    output = tfa.layers.WeightNormalization(
    layers.Dense(
                units = 32,
                activation='relu',
                kernel_initializer = "lecun_normal"))(output) 
    output = layers.Dropout(0.4)(layers.Concatenate()([embed, hidden, output]))
    output = tfa.layers.WeightNormalization(
    layers.Dense(
                units = 32, 
                activation = 'relu',
                kernel_initializer = "lecun_normal"))(output)
    
    conv_outputs = layers.Dense(
                units = 9, 
                activation ='softmax',
                kernel_initializer ="lecun_normal")(output)
    
    model = Model(conv_inputs,conv_outputs)
    
    return model

In [12]:
import datetime;
import time;

def do_run(run_number):
    print(datetime.datetime.now())
    start = time.time()

    oof_NN_a = np.zeros((train.shape[0],9))
    pred_NN_a = np.zeros((test.shape[0],9))

    N_FOLDS = 40
    EPOCH = 25

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True)

    for fold, (tr_idx, ts_idx) in enumerate(skf.split(train,train.iloc[:,-1])):
        #print(f"\n ====== TRAINING FOLD {fold} =======")

        X_train = train.iloc[:,1:-1].iloc[tr_idx]
        y_train = targets.iloc[tr_idx]
        X_test = train.iloc[:,1:-1].iloc[ts_idx]
        y_test = targets.iloc[ts_idx]

        K.clear_session()

        #print("\n-----Convolution model Training----\n")
        model_conv = conv_model()

        model_conv.compile(loss='categorical_crossentropy', 
                                optimizer= 'adam', #keras.optimizers.Adam(learning_rate=2e-4), 
                                metrics=custom_metric)
        model_conv.fit(X_train, y_train,
                  batch_size = 256, epochs = EPOCH,
                  validation_data=(X_test, y_test),
                  callbacks=[es, plateau],
                  verbose = 0)

        pred_a = model_conv.predict(X_test) 
        oof_NN_a[ts_idx] += pred_a 
        score_NN_a = log_loss(y_test, pred_a)
        print(f"RUN {run_number} - FOLD {fold} - Score: {score_NN_a}")
        pred_NN_a += model_conv.predict(test.iloc[:,1:]) / N_FOLDS 

    score_a = log_loss(targets, oof_NN_a)
    print(f"=== FINAL SCORE CONVOLUTION MODEL : {score_a} ===")
    print(datetime.datetime.now())
    minutes = (time.time() - start) / 60
    print('minutes', minutes, '\n')
    
    return pred_NN_a
    

In [13]:
n_runs = 11
ensemble = do_run(1)

for i in range(2, n_runs+1):
    for_result = do_run(i)
    ensemble = ensemble + for_result
    
ensemble = ensemble / n_runs

2021-06-25 22:51:38.975842
RUN 1 - FOLD 0 - Score: 1.744124080070108
RUN 1 - FOLD 1 - Score: 1.7521766049796716
RUN 1 - FOLD 2 - Score: 1.7327886724840849
RUN 1 - FOLD 3 - Score: 1.7430086870480328
RUN 1 - FOLD 4 - Score: 1.7532614020533859
RUN 1 - FOLD 5 - Score: 1.7398305557221174
RUN 1 - FOLD 6 - Score: 1.7443677926242351
RUN 1 - FOLD 7 - Score: 1.7359587298780679
RUN 1 - FOLD 8 - Score: 1.750457779929787
RUN 1 - FOLD 9 - Score: 1.7450280501112343
RUN 1 - FOLD 10 - Score: 1.7429215138390661
RUN 1 - FOLD 11 - Score: 1.7521596642520278
RUN 1 - FOLD 12 - Score: 1.7381040653754025
RUN 1 - FOLD 13 - Score: 1.7520060794629158
RUN 1 - FOLD 14 - Score: 1.7586145431257785
RUN 1 - FOLD 15 - Score: 1.743232160335034
RUN 1 - FOLD 16 - Score: 1.7516227200016379
RUN 1 - FOLD 17 - Score: 1.744360311959684
RUN 1 - FOLD 18 - Score: 1.7597689539324493
RUN 1 - FOLD 19 - Score: 1.746646310981363
RUN 1 - FOLD 20 - Score: 1.7484886681072414
RUN 1 - FOLD 21 - Score: 1.746203337899968
RUN 1 - FOLD 22 - Sco

RUN 5 - FOLD 15 - Score: 1.7492778582334518
RUN 5 - FOLD 16 - Score: 1.7529509950898587
RUN 5 - FOLD 17 - Score: 1.7560066559437661
RUN 5 - FOLD 18 - Score: 1.752504641218111
RUN 5 - FOLD 19 - Score: 1.7346474234119058
RUN 5 - FOLD 20 - Score: 1.7502057047639041
RUN 5 - FOLD 21 - Score: 1.743115379293263
RUN 5 - FOLD 22 - Score: 1.7620259629361332
RUN 5 - FOLD 23 - Score: 1.7375706924900411
RUN 5 - FOLD 24 - Score: 1.753541583974287
RUN 5 - FOLD 25 - Score: 1.754403003410995
RUN 5 - FOLD 26 - Score: 1.7406675810918213
RUN 5 - FOLD 27 - Score: 1.7629048256285489
RUN 5 - FOLD 28 - Score: 1.7432223049223423
RUN 5 - FOLD 29 - Score: 1.7375956851605325
RUN 5 - FOLD 30 - Score: 1.740901625077054
RUN 5 - FOLD 31 - Score: 1.7494987190958113
RUN 5 - FOLD 32 - Score: 1.7419863809237257
RUN 5 - FOLD 33 - Score: 1.747476669292897
RUN 5 - FOLD 34 - Score: 1.745952110875398
RUN 5 - FOLD 35 - Score: 1.7497407183978706
RUN 5 - FOLD 36 - Score: 1.742912404690683
RUN 5 - FOLD 37 - Score: 1.7513015386361

RUN 9 - FOLD 31 - Score: 1.7416433317542077
RUN 9 - FOLD 32 - Score: 1.7465220336014404
RUN 9 - FOLD 33 - Score: 1.7572523491378873
RUN 9 - FOLD 34 - Score: 1.7440554220840334
RUN 9 - FOLD 35 - Score: 1.7421367367755622
RUN 9 - FOLD 36 - Score: 1.757068282944709
RUN 9 - FOLD 37 - Score: 1.7491070446319879
RUN 9 - FOLD 38 - Score: 1.750327733590454
RUN 9 - FOLD 39 - Score: 1.7375551483511924
=== FINAL SCORE CONVOLUTION MODEL : 1.746449067625179 ===
2021-06-26 03:54:21.403378
minutes 33.10527095794678 

2021-06-26 03:54:21.412330
RUN 10 - FOLD 0 - Score: 1.7432564385049045
RUN 10 - FOLD 1 - Score: 1.7594447200506926
RUN 10 - FOLD 2 - Score: 1.7525176377851517
RUN 10 - FOLD 3 - Score: 1.7435624278135597
RUN 10 - FOLD 4 - Score: 1.7397539880353958
RUN 10 - FOLD 5 - Score: 1.735697718194872
RUN 10 - FOLD 6 - Score: 1.7450859208950773
RUN 10 - FOLD 7 - Score: 1.74695926027894
RUN 10 - FOLD 8 - Score: 1.7456078631140293
RUN 10 - FOLD 9 - Score: 1.7486401695534588
RUN 10 - FOLD 10 - Score: 1.7

In [14]:
submission = pd.read_csv("./sample_submission.csv")
submission['Class_1']=ensemble[:,0]
submission['Class_2']=ensemble[:,1]
submission['Class_3']=ensemble[:,2]
submission['Class_4']=ensemble[:,3]
submission['Class_5']=ensemble[:,4]
submission['Class_6']=ensemble[:,5]
submission['Class_7']=ensemble[:,6]
submission['Class_8']=ensemble[:,7]
submission['Class_9']=ensemble[:,8]

submission.to_csv("submission_cnn_ensemble.csv", index=False)